<div align="center">
    
# ΠΑΝΕΠΙΣΤΗΜΙΟ ΚΡΗΤΗΣ
# Τμήμα Επιστήμης Υπολογιστών

## HY-215: Εφαρμοσμένα Μαθηματικά για Μηχανικούς
## Εαρινό Εξάμηνο 2022-2023
### Διδάσκοντες: Γ. Στυλιανού, Γ. Καφεντζής
    
## Ανάθεση: 7-4-2023
## Παράδοση (ηλεκτρονικά, όπως αναφέρεται στην ιστοσελίδα του μαθήματος): 
## Τρίτη, 25-4-2023, 23:59:59
<h1><center>Άσκηση 1 [Υποχρεωτική]</center></h1>

In [ ]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import lfilter
import IPython

Κατά την παραγωγή και καταγραφή ήχου σε ένα χώρο όπου υπάρχουν πολλές ανακλάσεις, εμπόδια, κλπ., το σήμα που καταγράφεται αποτελείται από ένα άθροισμα πολλών διαφορετικών "εκδόσεων" (καθυστερήσεων, εξασθενήσεων κλπ) του σήματος που προσπαθούμε να καταγράψουμε. Όλες αυτές οι διαφορετικές "εκδόσεις" που επιστρέφουν μαζί στο μικρόφωνο μπορούν να θεωρηθούν ότι αποτελούν μια αντήχηση του επιθυμητού σήματος. Μπορούμε να μοντελοποιήσουμε την εφαρμογή μιας απλής αντήχησης ως ένα ΓΧΑ σύστημα, το οποίο περιγράφεται από τη σχέση:
$$\large y(t) = x(t) + ax(t-t_d) $$
με $a$ το πλάτος μιας αντήχησης και $t_d$ τη θέση της στο χρόνο, δηλ. τη χρονική στιγμή που εμφανίζεται στο ηχογραφημένο σήμα. Το παραπάνω σύστημα ουσιαστικά δίνει ως έξοδο το σήμα που θέλουμε να καταγράψουμε μαζί με μια έκδοσή του, καθυστερημένη κατά $t_d$ και με εξασθενημένο πλάτος κατά $a$.

Θα μπορούσαμε να προσθέσουμε κι άλλα αντίγραφα αυτής της αντήχησης σε διαφορετικές χρονικές στιγμές και με διαφορετικούς συντελεστές. Όπως μπορείτε εύκολα να καταλάβετε, ένα τέτοιο σύστημα θα είναι της μορφής:
$$\large y(t) = x(t) + \sum_{i=1}^N a_i x(t-t_i)$$
Δεν είναι δύσκολο να βρείτε την κρουστική απόκριση αυτού του ΓΧΑ συστήματος: θέτουμε όπου $x(t)$ το $\delta(t)$ και έτσι παίρνουμε
$$\large h(t) = \delta(t) + \sum_{i=1}^N a_i \delta(t-t_i)$$

Θα υλοποιήσουμε το παραπάνω σύστημα παραγωγής ηχούς επάνω σε ένα οποιοδήποτε ηχητικό σήμα εισόδου συμπληρώνοντας την επόμενη συνάρτηση στην Python:

In [ ]:
def apply_filter_tostudents(signal, times, attenuations, fs):
    """
    1) υπολογίζει και επιστρέφει την κρουστική απόκριση h του ΓΧΑ συστήματος
    2) φιλτράρει το σήμα εισόδου signal και παράγει την έξοδο y_echo
    :param signal: σήμα εισόδου που θέλετε να βάλετε αντήχηση
    :param times: διάνυσμα που περιέχει τις χρονικές στιγμές - σε δευτερόλεπτα - που θέλουμε να ξεκινά μια αντήχηση επάνω στο σήμα εισόδου
    :param attenuations: διάνυσμα που περιέχει το πλάτος της κάθε αντήχησης στις αντίστοιχες χρονικές στιγμές που έχετε ορίσει στο διάνυσμα times επάνω
    :param fs: συχνότητα δειγματοληψίας του σήματος, επιστρέφεται από την wav.read
    :return: y_echo = έξοδος του ΓΧΑ συστήματος (σήμα με αντήχηση), h = κρουστική απόκριση του ΓΧΑ συστήματος που υλοποιεί την αντήχηση
    """
    h = np.zeros(shape=len(signal))
    samples = np.zeros(shape=len(times))
    h[0] =  signal[0]

    for i in range(len(times)):
        samples[i] =  times[i] * fs
        h[int(samples[i])] = attenuations[i] * signal[(i - times[i])]

    y_echo = np.convolve(signal, h, mode='full') #at first it took 2 and half minutes and i had extra silence
                                                # and i read the man of the comvolve funtion and i tried the modes
                                                # (to be honest i am not complitely sure what mode should i use)
                                                #the defult full takes longer and creates some empty seconds at the end
                                                #but same cuts some seconds and valid seams not to work
                                                #so i guess the best option is the defult full allthough it takes a lot of time.

    return y_echo, h

Μια μικρή επεξήγηση για τη συνάρτηση που πρέπει να συμπληρώσετε. Επειδή όλα τα σήματα που επεξεργαζόμαστε στον υπολογιστή είναι διακριτού χρόνου, δηλ. ορισμένα για συγκεκριμένες χρονικές τιμές (και όχι για κάθε t), εσείς πρέπει αρχικά να ορίσετε τις τιμές του διανύσματος times που θέλετε να ακούγεται η ηχώ (σε δευτερόλεπτα), και να μετατρέψετε κάθε τιμή του διανύσματος αυτού σε ακέραιες τιμές, δηλ. σε δείγματα. Αυτό γίνεται αν λάβετε υπόψη σας ότι η συχνότητα δειγματοληψίας $f_s$ ενός σήματος σας λέει ότι σε ένα δευτερόλεπτο ηχογράφησης έχουν παρθεί και αποθηκευτεί $f_s$ δείγματα (τιμές) του σήματος στον υπολογιστή. ΄Αρα, για παράδειγμα, η χρονική στιγμή $t_0 = 0.5$ s αντιστοιχεί στο δείγμα διακριτού χρόνου $f_s/2$. Σε ποιά δείγματα αντιστοιχούν οι δικές σας χρονικές στιγμές της ηχούς που ορίσατε στο διάνυσμα $\textrm{times}$;

Μπορείτε να χρησιμοποιήσετε ένα οποιοδήποτε σήμα φωνής/μουσικής σε μορφή .WAV για να ελέγξετε τη λειτουργία του συστήματός σας. Απλά φροντίστε να μην είναι πολύ μεγάλης διάρκειας για να μην κρασάρετε το πρόγραμμα. Για δική σας ευκολία, σας δίνονται δυο αρχεία μαζί με τον κώδικα. Μια εντολή για να φορτώσετε ένα .WAV σήμα με όνομα "furelise.wav" μέσω της SciPy είναι:

In [ ]:
sr, data = wavfile.read('furelise.wav')
IPython.display.Audio(data, rate=sr)

Στη συνέχεια μπορούμε να χρησιμοποιήσουμε την παραπάνω συνάρτηση για να εφαρμόσουμε αντήχηση τη χρονική στιγμή $t=1$ sec, με πλάτος $0.5$. Αυτό σημαίνει ότι το παραγόμενο από το σύστημα σήμα θα ακούσουμε αντήχηση την προαναφερθείσα χρονική στιγμή αλλά η "έντασή" της θα είναι στο μισό της "έντασης" της αρχικής ηχογράφησης.

In [ ]:
reverb, h = apply_filter_tostudents(signal=data, times=[1], attenuations=[.5], fs=sr)

Ας το ακούσουμε.

In [ ]:
IPython.display.Audio(reverb, rate=sr)